In [ ]:
!pip install wandb --upgrade

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle


In [ ]:
! cp kaggle.json ~/.kaggle/
!kaggle datasets download -d jangedoo/utkface-new
!unzip -qq utkface-new.zip

 98% 323M/331M [00:02<00:00, 177MB/s]
100% 331M/331M [00:02<00:00, 165MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import wandb

In [ ]:

import torch
import  torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import os
import pandas as pd
import numpy as np
import cv2
from torch.utils.data import TensorDataset, Dataset

In [ ]:
sweep_config = {
  "method": "random",
  "metric": {
    "name": "loss",
    "goal": "minimize" 
  },
    
  "parameters": {
      "learning_rate":{
          "min": 0.0001,
          "max": 0.01
      },
      "epochs": {
          'values': [1,2]
      },   
  }
    
}
sweep_id = wandb.sweep(sweep_config, project="transfer learning and fine tuning")
sweep_id

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 5h20oyes
Sweep URL: https://wandb.ai/amirrooholamini/transfer%20learning%20and%20fine%20tuning/sweeps/5h20oyes


'5h20oyes'

In [ ]:
images = [] # X
ages = [] # Y

for image_name in os.listdir('crop_part1')[0: 9000]:
    parts = image_name.split('_')
    ages.append(int(parts[0]))
    
    image = cv2.imread(f'crop_part1/{image_name}')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)

images = pd.Series(images, name='Images')
ages = pd.Series(ages, name='Ages')

df = pd.concat([images, ages], axis=1)
# df.head()

In [ ]:
under_4 = []

for i in range(len(df)):
  if df['Ages'].iloc[i] <= 4:
    under_4.append(df.iloc[i])

under_4 = pd.DataFrame(under_4)
under_4 = under_4.sample(frac=0.3)

up_4 = df[df['Ages'] > 4]

df = pd.concat([under_4, up_4])

In [ ]:
df = df[df['Ages'] < 90]

In [ ]:
X = []
Y =[]

for i in range(len(df)):
  df['Images'].iloc[i] = cv2.resize(df['Images'].iloc[i], (224, 224))

  X.append(df['Images'].iloc[i])
  Y.append(df['Ages'].iloc[i])

X = np.array(X)
Y = np.array(Y)

X = X.reshape((-1, X.shape[3], X.shape[1], X.shape[2]))
Y = Y.reshape(Y.shape[0], 1)
print(X.shape, Y.shape,X[0].shape)

X = torch.tensor(X)
Y = torch.tensor(Y)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


(7538, 3, 224, 224) (7538, 1) (3, 224, 224)


In [ ]:
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform):

        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]
        y = self.tensors[1][index]

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)



transform = transforms.Compose([
    torchvision.transforms.ToPILImage(),
    transforms.RandomRotation(10),
    # transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0), (1))
])


train_dataset = CustomTensorDataset(tensors=(X, Y), transform=transform)

In [ ]:
torch.manual_seed(0)

train_dataset_size = int(0.8 * len(train_dataset))
test_dataset_size = len(train_dataset) - train_dataset_size

train_data, _ = torch.utils.data.random_split(train_dataset, [train_dataset_size, test_dataset_size])
train_data = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=16)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
# Hyper params
counter = 0

def train():
  global counter
  counter += 1
  print('training ...')
  config_defaults = {
        'epochs': 5,
        'learning_rate': 0.001
    }
  wandb.init(config=config_defaults)
  config=wandb.config

  model = torchvision.models.resnet50(pretrained=True)
  in_features = model.fc.in_features
  model.fc = nn.Linear(in_features, 1)

  counter = 0
  for child in model.children():
    counter += 1
    if(counter < 7):
      for params in child.parameters():
        params.requires_grad = False

  model.to(device)
  wandb.watch(model)

  optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
  loss_function = nn.L1Loss()

  model.train()

  for epoch in range(config.epochs):
    train_loss = 0.0
    train_acc = 0.0
    for images, labels in tqdm(train_data):

      images, labels = images.to(device), labels.to(device)

      optimizer.zero_grad()
      images = images.float()
      preds = model(images)

      loss = loss_function(preds,  labels.float())
      loss.backward()

      optimizer.step()

      train_loss += loss

    total_loss = train_loss / len(train_data)
    print(f"Epoch: {epoch+1}, Loss: {total_loss}")

    wandb.log({'epochs':  epoch + 1,'loss': total_loss,})

  torch.save(model.state_dict(),'weights_config_' + counter + ".pth")

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
# Inference

index = 1

import cv2 as cv
import numpy as np

model = torchvision.models.resnet50(pretrained=False)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 1)
model.load_state_dict(torch.load('weights_config_' + index + ".pth"))

inference_transform = transforms.Compose([
     transforms.ToTensor(),
     torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

model.to(device)
model.train(False)
model.eval()

img = cv.imread('three.png')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img = cv.resize(img,(64,64))
tensor = inference_transform(img).unsqueeze(0).to(device)
predictions = model(tensor).cpu().detach().numpy()
print(np.argmax(predictions, axis=1))